In [2]:
import pandas as pd

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#This code is for combining certain data from the necessary csv files into a single dataframe (complete)
pd.set_option('display.max_colwidth', None)

movies_full = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})

keywords = pd.read_csv('newdata/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
credits = pd.read_csv("newdata/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})

complete =  pd.merge(movies_full, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


#new:
#userIds are in order
# every_id = list(unique(list(complete["userId"])))
#userIds are out of order
# sample_ids  = random.sample(every_id, 1000)
# completeNew = pd.DataFrame()
#This is a very expensive task...
#it is possile to choose a subset of users from here instead of
#ibcluing  the entire set of users
# for user in sample_ids:
#     completeNew = pd.concat([completeNew, complete.loc[complete["userId"] == user]])

#new:
# complete = complete.sample(frac=1, random_state = seed_int, axis =0)
#this is not the desired behavior
#the users ids need to show up in a true random order
# complete = complete.groupby(by = "userId", sort = False, group_keys = True).apply(lambda x: x)
#this is omitted since the values should not be sorted by userId just grouped by userId

complete = complete.sort_values(by = 'userId')

complete  = complete.dropna()

complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]


print(complete.head())

# f = open("test_dicts.txt", "w", encoding="utf-8")
# f.write(str(list(complete["tagline"])))
# f.write(str(list(complete["overview"])))
# f.close()


        userId    id rating               title  \
8830418      1  2959    4.0      License to Wed   
1690638      1  1968    4.0       Fools Rush In   
2343877      1  2762    4.5  Young and Innocent   
2943547      1   147    4.5       The 400 Blows   
8434697      1  1246    5.0        Rocky Balboa   

                                                                                                genres  \
8830418                                                                 [{'id': 35, 'name': 'Comedy'}]   
1690638  [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]   
2343877                                     [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}]   
2943547                                                                  [{'id': 18, 'name': 'Drama'}]   
8434697                                                                  [{'id': 18, 'name': 'Drama'}]   

                                                        

In [14]:
import ast
import random

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#used to filter out the rows of data with empty entries
def condition(array):
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    #this is probably not needed due to the dropNa function used above...
    # length = len(array[8])
    # if(array[8][length-4:]=="<NA>"):
    #     return False
    # length = len(array[9])
    # if(array[9][length-4:]=="<NA>"):
    #     return False 
    return True


#used to extract names
def populate_names(item):
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        cnt += 1
    return names

#extract data from row of complete_array
def provide_data(array):
    movie_data = []
    movie_data.append(int(array[0]))
    movie_data.append(int(array[1]))
    movie_data.append(float(array[2]))
    movie_data.append(array[3])  

    movie_data.append(populate_names(array[4]))
    movie_data.append(populate_names(array[5]))
    movie_data.append(populate_names(array[6]))
    movie_data.append(populate_names(array[7]))

    movie_data.append(str(array[8]))
    movie_data.append(str(array[9]))
    return movie_data
    


#new method
list_of_user_ids = list(complete["userId"].unique())

counts = complete['userId'].value_counts()
gaps = []
for id in list_of_user_ids:
    gaps.append(counts[id])

complete_array = complete.to_numpy()

#old method...
# gaps = []
# size = 0
# list_of_user_ids = []
# last_id  = -1
# past_first_it = False


# for row in complete_array:
#     if(row[0]!= last_id):
#         list_of_user_ids.append(row[0])
#         last_id = row[0]
#         if(past_first_it ==True):
#             gaps.append(size)
#             size =0 
#     size+=1
#     past_first_it = True

# #there is always a gap for the last iteration
# gaps.append(size)



index  = 0

user_to_data = []
#this is the total number of users in the whole dataset
total_nof_users = len(list_of_user_ids)
#this is the number of desired users before filtering
#note: set back to 40000
desired_nof_users_before_filter = 40000

avg = 0
cnt = 0


#instead of using this random pass generaion 
#the users can be picked randomly before hand and replace list_of_user_ids[i]
#will index still work???

#populate user_to_data from complete_array
for i in range(0, total_nof_users):
    #generate a random float to determine a pass for the user
    if (random.random()<float(desired_nof_users_before_filter/total_nof_users)):
        #user_to_data[list_of_user_ids[i]] = []
        user_to_data.append([])
        last_index = len(user_to_data) -1
        for j in range(index, len(complete_array)):
            if complete_array[j][0] == list_of_user_ids[i]:
                #condition is checked for complete_array[j]
                if(condition(complete_array[j])):
                    #this is where data is tranformed
                    transformed = provide_data(complete_array[j])
                    #is copy needed here???
                    user_to_data[last_index].append(transformed)

                #last iteration
                if (j==len(complete_array)-1):
                    if (len(user_to_data[last_index])<50 or len(user_to_data[last_index])>75):
                        del user_to_data[last_index]       
            else:
                avg += len(user_to_data[last_index])
                cnt+=1
                #this condition can be tweaked for better accuracy
                #len(user_to_data[list_of_user_ids[i]])<50 or len(user_to_data[list_of_user_ids[i]])>75
                if (len(user_to_data[last_index])<50 or len(user_to_data[last_index])>75):
                    del user_to_data[last_index]  
                #note: changed from (index = j+1)
                index = j
                break
        #note, problem: provide data is run in right order
        #but a user can be deleted and then the order can be messed up    
    else:
        #every iteration, index starts at first data point of the next user
        index += gaps[i]


#runtime test, go through user_to_data and re-index the users in list order
for i in range(len(user_to_data)):
    for j in range(len(user_to_data[i])):
        user_to_data[i][j][0] = i



#needs to be sure that there are enough users after the condiiton
print(len(user_to_data))

#average number of ratings per users
print(float(avg/cnt))

#18 minutes for 2599 users

#test with dictionaries:
# 71
#runtime: 30.679282868525895

#test without dictionaries:
# 63
#runtime: 30.422131147540984

#18 minutes for 2611 users


2575
29.902903104184993


In [15]:
#save in a file so that cells below can run without running this cell and above

#question: would renaming the user ids as indexes in their order be helpful???

import csv

with open("constructedData/constructedData.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for i in range(len(user_to_data)):
        writer.writerows(user_to_data[i])

In [17]:
#this is a starting point if the data is already saved to the constructedData.csv file
import csv

data_list =[]

with open("constructedData/constructedData.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    data_list = list(csv_reader)

data_list = data_list[1:]


In [18]:
import random

#try removing unecessay copying...

#seed for consistent results across runtime
# seed_int = 1
# random.seed(seed_int)

#user to data rows 
user_to_data = []
user_to_data_train = []
user_to_data_test = []
user_id = -1

#note: works when row[0] is also an index
for row in data_list:
    if (row[0]!=user_id):
        user_id = row[0]
        user_to_data.append([row])
    else:
        user_to_data[int(row[0])].append(row)


#this can be tweaked...
for i in range(500):
    index = random.randint(0, len(user_to_data)-1)
    user_to_data_train.append(user_to_data[index])
    del user_to_data[index]


#for test data to be used later...
for i in range(100):
    index = random.randint(0, len(user_to_data_train)-1)
    user_to_data_test.append(user_to_data_train[index])
    del user_to_data_train[index]


del user_to_data


In [19]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
import random
import json
from ordered_set import OrderedSet



#seed for consistent results across runtime
# seed_int = 1
# random.seed(seed_int)


#check if there are variables that can be removed!!!
class user_type_vars():
    def __init__(self):
        #user to movie_id to a list of all the words in that movie (copies allowed)
        self.user_to_movie_id_to_corpus = [] 

        #user to movie_id to its rating
        self.user_to_movie_id_to_rating = [] 

        #for each user, includes a random choice from all the movies the user watched to be the target
        self.user_to_target_movie_id = [] 

        #for each user, the index is the placement of the target movie in the entire set of train movies inorder
        self.user_to_target_index_full = [] 

        #for each user, a set of all the words in all the movies the user watched 
        self.user_to_words_in_order  = [] 

        #for each user, the word counts for movies that they watched
        self.user_to_word_counts  = [] 
        self.user_to_word_counts_transformed = []

        #The word counts of the target movie for each user
        self.user_to_target_word_counts  = [] 
        self.user_to_target_word_counts_transformed = []

        #for each user, includes all the ratings of movies the user has rated with a -1 marked as the target rating
        self.user_to_ratings  = []

        #for each user, includes ratings for all the movies in the entire train set (missing ratings are filled in)
        self.user_to_ratings_full = [] 
        self.user_to_ratings_full_transform = []

        #model features x
        self.feature_1 = []
        self.feature_2 = []
        self.feature_3 = []

        #output feature y
        self.user_to_target_rating  = [] 


#need to think about the order these variables are used to improve memory perf...
#goal: minimize the window between when a variable was populated and used


#user_to_movie_id_to_corpus:
#1,2, 3, 4

#user_to_movie_id_to_rating:
#1, 3, after 5, 6

#user_to_target_movie_id:
#1, 3, 4, after 5, 6

#user_to_target_index_full:
#6,7

#user_to_words_in_order:
#2,4
#this seems optimal

#user_to_word_counts:
#4,5,

#user_to_word_counts_transformed:
#4,5,

#user_to_target_word_counts:
#4,5

#user_to_target_word_counts_transformed:
#4,5

#user_to_ratings: (the combination of 4 and 5 streamline this this)
#3,5

#user_to_ratings_full:
#6,7

#user_to_ratings_full_transform:
#6,7

#needed until the end:
#self.features...
#self.user_to_target_rating


#goal: minimize the window between when a vraible was populated and used
#or remove intermediate variables 


# first observation:
# these variables are populated and used in sucession
# user_to_word_counts:
# user_to_word_counts_transformed:
# user_to_target_word_counts:
# user_to_target_word_counts_transformed:

# the sole purpose of 4 and 5 is to created a cossine similairty list
# so populate 4 and 5 can be combined


#idea:
#exchange places of 2 and 3


#idea:
#after the switch is populate 3 even necessary ???



        
train_users = user_type_vars()
test_users = user_type_vars()



#no changes needed for these vars below...
movie_id_to_ratings_total = dict() #all ratings for each movie
movies_in_order = OrderedSet() #all movies in order
movie_id_to_average_rating_train = dict() #average movie rating amoung all users for each movie
user_to_average_rating = []
overall_sum_train = 0 #used to compute the overall_average
overall_counts_train = 0 #used to compute the overall_average


wnl = WordNetLemmatizer()


def populate_1(user_to_data, user_to_movie_id_to_corpus, user_to_movie_id_to_rating,
            movie_id_to_ratings, overall_sum, overall_counts, user_to_target_movie_id):
    for i in range(len(user_to_data)):
        movie_id_to_words_temp = dict()
        movie_id_to_rating_temp = dict()
        cnt = 0
        total =0
        rand_int = random.randint(0, len(user_to_data[i])-1)
        for movie_data in user_to_data[i]:
            if cnt == rand_int:    
                user_to_target_movie_id.append(movie_data[1])
            else:
                overall_sum += float(movie_data[2])
                overall_counts += 1
                total += float(movie_data[2])
            
            if movie_data[1] in movie_id_to_ratings.keys():
                movie_id_to_ratings[movie_data[1]].append(float(movie_data[2]))
            else:
                movie_id_to_ratings[movie_data[1]] = [float(movie_data[2])]

            movie_string = ""
            #avoid the first three data points (user id, movieid, and rating)
            #use only the text data
            for index in range (3,len(movie_data)):
                if(index!= len(movie_data)-1):
                    movie_string+= movie_data[index]+" "
                else:
                    movie_string+= movie_data[index]

            #need to try with and without lematization...
            cleaned = remove_stopwords(movie_string)
            cleaned = [wnl.lemmatize(word) for word in cleaned.split(" ")]
            cleaned = [word[:-1] for word in cleaned if word.endswith(".")] + [word for word in cleaned if not word.endswith(".")]
            #is copy really needed with this scope ???
            movie_id_to_words_temp[movie_data[1]] = cleaned
            movie_id_to_rating_temp[movie_data[1]] = float(movie_data[2])
            movies_in_order.add(movie_data[1])
            cnt+=1
        user_to_average_rating.append(float(total/(cnt-1)))
        #is copy really needed with this scope ???
        user_to_movie_id_to_corpus.append(movie_id_to_words_temp)
        #is copy really needed with this scope ???
        user_to_movie_id_to_rating.append(movie_id_to_rating_temp)
    return overall_sum, overall_counts


overall_sum_train, overall_counts_train = populate_1(user_to_data_train, train_users.user_to_movie_id_to_corpus, 
                                                   train_users.user_to_movie_id_to_rating,movie_id_to_ratings_total,
                                                   overall_sum_train, overall_counts_train, train_users.user_to_target_movie_id)

#this is used when there are no usable ratings for a movie
overall_average_train = float(overall_sum_train/overall_counts_train)

populate_1(user_to_data_test, test_users.user_to_movie_id_to_corpus, test_users.user_to_movie_id_to_rating,
            dict(), 0, 0, test_users.user_to_target_movie_id)


#movie_id_to_average_rating_train is used by both the test and train set
for movie in movie_id_to_ratings_total.keys():
    temp = 0
    for rating in movie_id_to_ratings_total[movie]:
        temp +=rating
    movie_id_to_average_rating_train[movie] = float(temp/len(movie_id_to_ratings_total[movie]))


#used to see what the text data looks like...

# not applicable with dict to list change...
# file = open("test_dicts_1.txt", 'w', encoding="utf-8")
# file.write(json.dumps(user_to_movie_id_to_corpus_train))
# file.close()

# file = open("test_dicts_2.txt", 'w', encoding="utf-8")
# file.write(json.dumps(user_to_movie_id_to_corpus_test))
# file.close()

del user_to_data_train
del user_to_data_test


In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from ordered_set import OrderedSet
from collections import Counter



#what if populate 1 2 3 and 4 were combined???


def populate_2(user_to_movie_id_to_corpus, user_to_words_in_order):
    for i in range(len(user_to_movie_id_to_corpus)):
        user_to_words_in_order.append(OrderedSet())
        for movie_id in user_to_movie_id_to_corpus[i].keys():
            for word in user_to_movie_id_to_corpus[i][movie_id]:
                user_to_words_in_order[i].add(word)

populate_2(train_users.user_to_movie_id_to_corpus,  train_users.user_to_words_in_order)
populate_2(test_users.user_to_movie_id_to_corpus,  test_users.user_to_words_in_order)



def populate_3(user_to_movie_id_to_corpus, user_to_target_movie_id, user_to_movie_id_to_rating,
    user_to_target_rating, user_to_ratings):
    for i in range(len(user_to_movie_id_to_corpus)):
        temp = []
        cnt = 0
        for movie_id in user_to_movie_id_to_corpus[i].keys():
            if movie_id != user_to_target_movie_id[i]:
                temp.append(user_to_movie_id_to_rating[i][movie_id])
            else:
                #this signifies the ratings to be predicted by the model
                user_to_target_rating.append(user_to_movie_id_to_rating[i][movie_id])
                temp.append(-1)
            cnt+=1
        #is copy needed with this scope???
        user_to_ratings.append(temp)


populate_3(train_users.user_to_movie_id_to_corpus, train_users.user_to_target_movie_id, train_users.user_to_movie_id_to_rating,
            train_users.user_to_target_rating, train_users.user_to_ratings)
populate_3(test_users.user_to_movie_id_to_corpus, test_users.user_to_target_movie_id, test_users.user_to_movie_id_to_rating,
            test_users.user_to_target_rating, test_users.user_to_ratings)


#note: user to word counts needs to omit movies with no rating

def populate_4(user_to_word_counts_transformed, user_to_target_word_counts_transformed,
                user_to_movie_id_to_corpus, user_to_word_counts,
                user_to_target_movie_id, user_to_words_in_order, user_to_target_word_counts):
    for i in range(len(user_to_movie_id_to_corpus)):
        user_to_word_counts.append([])
        user_to_word_counts_transformed.append([])
        for movie_id in user_to_movie_id_to_corpus[i].keys():
            if movie_id != user_to_target_movie_id[i]:
                #idea: instead of using words in order use a set of words that are local to the user
                #this cuts time iterating over words_in_order
                #and cuts space by having templist be shorter
                #note: it also adds a bit of space by needing a list of words for each user
                temp_dict = Counter(user_to_movie_id_to_corpus[i][movie_id])
                temp_list = []
                avg = 0
                for word in user_to_words_in_order[i]:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                        avg+=temp_dict[word]
                    else:
                        temp_list.append(0)
                avg = float(avg/len(user_to_words_in_order[i]))
                #is copy needed with this scope???
                user_to_word_counts[i].append(temp_list)
                user_to_word_counts_transformed[i].append([x - avg for x in temp_list])
            else:

                temp_dict = Counter(user_to_movie_id_to_corpus[i][movie_id])
                temp_list = []
                avg = 0
                for word in user_to_words_in_order[i]:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                        avg+=temp_dict[word]
                    else:
                        temp_list.append(0)
                avg = float(avg/len(user_to_words_in_order[i]))
                #is copy needed with this scope???
                user_to_target_word_counts.append(temp_list)
                user_to_target_word_counts_transformed.append([x - avg for x in temp_list])


populate_4(train_users.user_to_word_counts_transformed, train_users.user_to_target_word_counts_transformed,
           train_users.user_to_movie_id_to_corpus, train_users.user_to_word_counts, train_users.user_to_target_movie_id,
       train_users.user_to_words_in_order, train_users.user_to_target_word_counts)

populate_4(test_users.user_to_word_counts_transformed, test_users.user_to_target_word_counts_transformed,
           test_users.user_to_movie_id_to_corpus, test_users.user_to_word_counts, 
       test_users.user_to_target_movie_id, test_users.user_to_words_in_order, test_users.user_to_target_word_counts)

#try omitting deletions...
del train_users.user_to_words_in_order
del train_users.user_to_movie_id_to_corpus
del test_users.user_to_words_in_order
del test_users.user_to_movie_id_to_corpus



#now for each user, use the user_to_index_full and find the most similair users by omitting that index across the
#sim_matrix



#collaboritive filtering idea
#data structures:
#need a user to movies to ratings dictionary (done) (only needed for train data)
#need a ordered set of all movies (done) (only needed for train data)

#need to transform it into a user to list of all movies with user ratings or otherwise filled in ratings with sutiable average
#(the list needs to be in a consistent order across users)
#to fill in the averages we need a movie to average rating dictionary
#(note: if no other user has rated the movie then fill it in with the overall movie average)
#standardize the data row wise (why not columnwise???)
#filter the users that have rated the movie to predict
#then use cossine similairity on this set to find the most similair user the the chosen user
#ignore the movie rating to predict with the cossine similarity function

#Note: this is an expensive task and the number of users may have to be truncated before running



#ideas: 
#idea1: 
#collaborative filtering:
#https://towardsdatascience.com/predict-movie-ratings-with-user-based-collaborative-filtering-392304b988af
#https://www.geeksforgeeks.org/user-based-collaborative-filtering/#
#https://www.youtube.com/watch?v=3ecNC-So0r4&ab_channel=CodeHeroku
#idea2: 
#note: the model can be scored based on how close a predition is to a threshold of .5 

#idea 3:
#there may be a replacement for cossine similarity:
#perhaps it is not properly matching similair people and similair movies 
#maybe there is a replacement function

#idea 4: 
#there should be more train and test users

#idea 5: (basic model)
#https://www.kaggle.com/code/muhammadayman/recommendation-system-using-cosine-similarity#Data-Cleaning



#10 minutes for 2500 users

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from statistics import mean


def predict(user, user_to_word_counts_transformed, user_to_target_word_counts_transformed,
            user_to_target_word_counts, user_to_word_counts, user_to_ratings):
    #before this operation, for each movie the average word count needs to be computed
    #and the transformed list of word counts subtracs the mean
    #note: in the next line you can siwtch beteen transformed and not to see if there are any boosts to perfromance

    cosine_sim = cosine_similarity(X = user_to_word_counts_transformed[user] ,Y = [user_to_target_word_counts_transformed[user]])
    
    #not sure if reshape is needed???
    #should test with and without reshape
    cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))

    ratings = [user_to_ratings[user][x] for x in range(len(user_to_ratings[user])) if user_to_ratings[user][x] != -1]


    #use the movie thats are most similair to the movie in question 

    #option 1: 
    # combined = zip(cosine_sim, ratings)
    # combined = sorted(combined, key=lambda x: x[0], reverse=False)
    # avg = 0
    # nof = 10.0
    # for i in range(int(nof)):
    #     avg += combined[i][1]
    # return float(avg/nof)

    #option 2:
    # avg = 0
    # for i in range(len(ratings)):
    #     avg += ratings[i]
    # return float(avg/len(ratings))

    #option 3: 
    combined = zip(cosine_sim, ratings)
    combined = sorted(combined, key=lambda x: x[0], reverse=True)
    avg = 0
    nof = 10.0
    for i in range(int(nof)):
        avg += combined[i][1]
    return float(avg/nof)




def populate_5(user_to_word_counts_transformed, user_to_target_word_counts_transformed,
               user_to_movie_id_to_rating, feature_1,
                user_to_target_word_counts, user_to_word_counts,
                user_to_ratings):
    for i in range(len(user_to_movie_id_to_rating)):
        feature_1.append(predict(i, user_to_word_counts_transformed, user_to_target_word_counts_transformed,
                                  user_to_target_word_counts, user_to_word_counts, user_to_ratings))



populate_5(train_users.user_to_word_counts_transformed, train_users.user_to_target_word_counts_transformed,
           train_users.user_to_movie_id_to_rating, train_users.feature_1,
                train_users.user_to_target_word_counts, train_users.user_to_word_counts,
                train_users.user_to_ratings)

populate_5(test_users.user_to_word_counts_transformed, test_users.user_to_target_word_counts_transformed,
           test_users.user_to_movie_id_to_rating, test_users.feature_1,
                test_users.user_to_target_word_counts, test_users.user_to_word_counts,
                test_users.user_to_ratings)

del train_users.user_to_word_counts
del test_users.user_to_word_counts
del train_users.user_to_word_counts_transformed
del test_users.user_to_word_counts_transformed
del train_users.user_to_target_word_counts
del test_users.user_to_target_word_counts
del train_users.user_to_target_word_counts_transformed
del test_users.user_to_target_word_counts_transformed
del train_users.user_to_ratings
del test_users.user_to_ratings


#note: for some reason feature 2 out performs feature 3 as a feature...

#populates feature_2_train
for i in range(len(train_users.user_to_movie_id_to_rating)): 
    if(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])==1):
        #this means that there is no other rating besides the first train rating
        train_users.feature_2.append(overall_average_train)
    else:
        #omit the rating the user in question made
        train_users.feature_2.append(float(((movie_id_to_average_rating_train[train_users.user_to_target_movie_id[i]]
                        *len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]]))
                        -train_users.user_to_movie_id_to_rating[i][train_users.user_to_target_movie_id[i]])
                        /(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])-1)))


#populates feature_2_test
for i in range(len(test_users.user_to_movie_id_to_rating)):
    if(test_users.user_to_target_movie_id[i] not in movie_id_to_ratings_total.keys()):
        #if the movie is not in the train set make a guess
        test_users.feature_2.append(overall_average_train)
    else:
        #only use the train data
        test_users.feature_2.append(movie_id_to_average_rating_train[test_users.user_to_target_movie_id[i]])



def populate_6(user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, 
               user_to_movie_id_to_rating, user_to_target_movie_id):
    for i in range(len(user_to_movie_id_to_rating)):
        ratings = []
        index = 0
        for movie_id in movies_in_order:
            #note: when this happens should rating be omitted???
            #it is omitted in another way later
            #why can it not be omitted now???
            if movie_id == user_to_target_movie_id[i]:
                user_to_target_index_full.append(index)
            if movie_id in user_to_movie_id_to_rating[i].keys():
                ratings.append(user_to_movie_id_to_rating[i][movie_id])
            elif movie_id in movie_id_to_average_rating_train.keys():
                #note: does not consider test ratings
                ratings.append(movie_id_to_average_rating_train[movie_id])
            else:   
                #note: does not consider test ratings
                ratings.append(overall_average_train)
            index +=1
        #is copy needed, considering the scope of ratings???
        #this behavior can be replace with append if they where lists
        user_to_ratings_full.append(ratings)
        user_to_ratings_full_transform.append([x - user_to_average_rating[i] for x in ratings])


populate_6(train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, train_users.user_to_target_index_full, train_users.user_to_movie_id_to_rating, train_users.user_to_target_movie_id)
populate_6(test_users.user_to_ratings_full_transform, test_users.user_to_ratings_full, test_users.user_to_target_index_full, test_users.user_to_movie_id_to_rating, test_users.user_to_target_movie_id)


del train_users.user_to_movie_id_to_rating
del test_users.user_to_movie_id_to_rating
del train_users.user_to_target_movie_id
del test_users.user_to_target_movie_id

#why is feature 3 outperformed by feature 2 ???

def populate_7(user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, feature_3):
    for i in range(len(user_to_ratings_full)):
        list_of_list_of_ratings = []
        sample_ratings = []
        for j in range(len(train_users.user_to_ratings_full)):
            if i != j:
                #note sure if this method is faster than full copy and deletion...
                #because there is a check that needs to be done for each item...
                sample_ratings.append(train_users.user_to_ratings_full[j][user_to_target_index_full[i]])
                ratings = [train_users.user_to_ratings_full_transform[j][x] 
                          for x 
                          in range(len(train_users.user_to_ratings_full_transform[j])) 
                          if x != user_to_target_index_full[i]]
                list_of_list_of_ratings.append(ratings)

        #note sure if this method is faster than full copy and deletion...
        #because there is a check that needs to be done for each item...
        user_ratings = [user_to_ratings_full_transform[i][x] 
                    for x 
                    in range(len(user_to_ratings_full_transform[i])) 
                    if x != user_to_target_index_full[i]]


        sim  = cosine_similarity(X = list_of_list_of_ratings, Y = [user_ratings])
        sim = np.reshape(sim,  (len(sim)))

        combined = zip(sim, sample_ratings)
        #note: orginally, reverse is set to true to get the most similair users
        #now trying reverse set to False
        #either way for some reason feature 2 outperfrom feature 3...
        combined = sorted(combined, key=lambda x: x[0], reverse = True)

        avg = 0
        nof = 10.0
        for k in range(int(nof)):
            avg+= combined[k][1]

        feature_3.append(float(avg/nof))
    

#populate feature_3_train and feature_3_test
populate_7(train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, train_users.user_to_target_index_full, train_users.feature_3)
populate_7(test_users.user_to_ratings_full_transform, test_users.user_to_ratings_full, test_users.user_to_target_index_full, test_users.feature_3)

del train_users.user_to_ratings_full_transform
del test_users.user_to_ratings_full_transform
del train_users.user_to_ratings_full
del test_users.user_to_ratings_full
del train_users.user_to_target_index_full
del test_users.user_to_target_index_full


#adjusted cossine simlairity
#https://stackoverflow.com/questions/40716459/choice-between-an-adjusted-cosine-similarity-vs-regular-cosine-similarity
#https://github.com/csaluja/JupyterNotebooks-Medium/blob/master/CF%20Recommendation%20System-Examples.ipynb?source=post_page-----ecbffe1c20b1--------------------------------
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1


#note: in the item-based collabortive filtering the means is implementaed item-wise
#this means with user-based collaboritve filtering the mean is implementad user-wise


#idea 1:
#use diferrent data structures...
#numpy arrays...
#https://www.geeksforgeeks.org/python-lists-vs-numpy-arrays/#
#https://stackoverflow.com/questions/29839350/numpy-append-vs-python-append
#https://www.geeksforgeeks.org/python-convert-list-to-python-array/


#idea 2:
#make use of both feature_2 and feature_3

#idea 3: would it be possible to convert many of the main dictionaries to lists or numpy arrays
#note: numpy arrays cant have variable number of items for a dimension
#but this can be over come by filling missing values
#this mean many matrices would be sparse
#there is also alot of appending to lists which is better for lists


#idea 4: would it be helpful to design my own data structures???

#idea 5: would it be helpful to have features like the average 10 closest in similairy and the 10 farthest in similarity???

#idea 6: before trying this there could be better data strcuture to use that would speed up the process



In [22]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score

#alternative to simply rerunning alaysis: run the entire code starting where random choice is implemented...

#earliest random choice is the test and train users
#this becomes averaged out with more users
#the next random choice is the random target movie for each user to be trained and tested against
#this can also be averged out with more users...
#lastly the there is randomness used in the model
#random state can be used to keep runs consistent
#but a number of random runs can be combined to see the real difference between including...
#different hyper parameters


#hyper paramaters: only the layers and the features are a hyper parameters at the moment
#def test_hyper_parameters():

# runs analysis a number of times and average the returned
# def average_results():

#returns the two average scores with and without rounding
# def analysis():




def test_parameters(layers, train_input_features, test_input_features):
    train_inputs = [list(pair) for pair in train_input_features]
    test_inputs = [list(pair) for pair in test_input_features]

    return average_results(20, layers, train_inputs, test_inputs)
    


def average_results(nof_runs, layers, train_inputs, text_inputs):
    no_rounding = 0
    rounding = 0
    for _ in range(nof_runs):
        pair = analysis(layers, train_inputs, text_inputs)
        no_rounding+=pair[0]
        rounding+=pair[1]
    return float(no_rounding/nof_runs), float(rounding/nof_runs)


def analysis(layers, train_inputs, test_inputs):
    #scale input features
    input_scalar = StandardScaler()
    train_inputs_scaled = input_scalar.fit_transform(train_inputs)

    #scale target values
    target_scalar = StandardScaler()
    true_rating_train_scaled = target_scalar.fit_transform(np.array(train_users.user_to_target_rating).reshape(-1, 1))
    true_rating_train_scaled = np.reshape(true_rating_train_scaled, len(true_rating_train_scaled))

    #build and train model
    reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
    reg.fit(train_inputs_scaled, true_rating_train_scaled)

    #show importance of different inputs features...
    results = permutation_importance(reg, train_inputs_scaled,true_rating_train_scaled)
    print(results["importances_mean"])

    #scale inputs features
    input_scalar = StandardScaler()
    test_inputs_scaled = input_scalar.fit_transform(test_inputs)

    #predict the scaled verison of ouptuts
    scaled_predictions = reg.predict(test_inputs_scaled)

    #change back into regular outputs to be tested with target scalar above...
    #not sure if this is correct???
    #would it be better to use a new scalar object???
    #need to know the mean and the std dev of the orginal predictions
    predictions = target_scalar.inverse_transform(np.array(scaled_predictions).reshape(-1, 1))

    #test with and without roundings...
    rounded_predictions = []
    for item in predictions:
        rounded_predictions.append(float(round(item[0]*2)/2.0))

    return(r2_score(test_users.user_to_target_rating, predictions), 
        r2_score(test_users.user_to_target_rating, rounded_predictions))


print(test_parameters((10,10,10), 
      zip(train_users.feature_1, train_users.feature_2),
      zip(test_users.feature_1, test_users.feature_2)))



# all (10,10,10):

# features 1 and 3 
# (0.18554102264617328, 0.16860136788605123)

# features 1 and 2
# (0.20158362399544255, 0.1773859572065005)

# features 1 and 3 with feature 3 reversed
# (0.21357221749346827, 0.20629067245119317)

# features 1 and 2 with feature 3 reversed
# (0.23359676025501463, 0.2268980477223428)

# feature 1 and 2 (with feature 1 reversed)
# (0.11148086956267429, 0.08668995334522105)

# feature 1 and 2 (with feature 1 complete)
# (0.4308803372158775, 0.43312216112837687)
# t2: (0.20265435431958068, 0.1869232087341802) 
# t3: (0.19338736134917242, 0.15702773818660246)
# need to try with more layers...
# (20,10,10)
# t4: (0.1963526265039616, 0.13345576142966786) 
# (20,15,10)
# t5: (0.1961748594852359, 0.1437342396667033)





[0.26144338 0.20693758]
[0.23599574 0.17306216]
[0.24793323 0.16964531]
[0.28303639 0.25911458]
[0.26362457 0.18855531]
[0.29633155 0.25954128]
[0.24254895 0.14437217]
[0.34149521 0.23429827]
[0.26492073 0.17915645]
[0.22740808 0.18277874]
[0.24104753 0.20289195]
[0.31190979 0.23754572]
[0.23101705 0.17039545]
[0.22029605 0.16711303]
[0.31540829 0.21983582]
[0.22340333 0.15789087]
[0.2284842  0.16540736]
[0.26537413 0.18416583]
[0.23419217 0.17812299]
[0.26700871 0.19593263]
(0.332180737747964, 0.3050224360707141)


In [23]:
#this cell is for observations and process descriptions


#How will the model work to predict ratings in the test data???
#for each test user a movie is randomly selected

#The prediction will be made with the word counts for the ratings by the user but using the
#the words_in_order of the train data for faster runtime (may change later)

#besides an average for a number of simailair movies the user has watched, the easy
#parameter is average rating since it only considers ratings from train data

#analysis without feature scaling...
#feature scaling:
#https://analyticsindiamag.com/why-data-scaling-is-important-in-machine-learning-how-to-effectively-do-it/#:~:text=Scaling%20the%20target%20value%20is,learn%20and%20understand%20the%20problem.&text=Scaling%20of%20the%20data%20comes,algorithms%20in%20the%20data%20set.
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1

#without scaling...
# reg = MLPRegressor(hidden_layer_sizes = (20,15,10), max_iter = 10000)
# train_inputs = []
# for feature_1, feature_2 in zip(feature_1_train, feature_3_train):
#     train_inputs.append([feature_1, feature_2])
# reg.fit(train_inputs, true_rating_train)
# results = permutation_importance(reg, train_inputs,true_rating_train)
# print(results["importances_mean"])
# test_inputs = []
# for feature_1, feature_2 in zip(feature_1_test, feature_3_test):
#     test_inputs.append([feature_1, feature_2])
# predictions = reg.predict(test_inputs)
# rounded_predictions = []
# for item in predictions:
#     rounded_predictions.append(float(round(item*2)/2.0))
# print(r2_score(true_rating_test, predictions))
# print(r2_score(true_rating_test, rounded_predictions))


# commented out for testing
# predicted = []
# for user in user_to_movie_id_to_corpus_train.keys(): 
#     if(len(movie_id_to_ratings[user_to_target_movie_id[user]])==1):
#         predicted.append(overall_average)
#     else:
#         predicted.append(float(((movie_id_to_average_rating[user_to_target_movie_id[user]]
#                         *len(movie_id_to_ratings[user_to_target_movie_id[user]]))
#                         -user_to_movie_id_to_rating[user][user_to_target_movie_id[user]])
#                         /(len(movie_id_to_ratings[user_to_target_movie_id[user]])-1)))
        
# nof_overall_avg = 0
# for item in predicted:
#     if item == overall_average:
#         nof_overall_avg += 1
# print(nof_overall_avg)
# print(r2_score(true, predicted))


#now for the user comparison logic (need user to list of movie ratings)
#fill in ratings that the user hasn't watched with the method above
#then cluster the users by their ratings

#note: agglomerative clustering might make more sense here since k-means has random init for centroids...
#note: to guess a new users rating requires that none of that users ratings have been used to train the model
#The data needs to be split into test and train before modeling the algorithm on the train data

#Training process:
#split data into test and train data
#proceed with train data...
#cluster movies by the tokens with range for k
#cluster users by the ratings with range for k and (fill in ratings for movies a users hasn't watched with some guess)
#guess: this can be obtained by clustering the movies that the user has watched...
#for each movie the user hasn't watched find the cluster that it belongs to with the highest possible k value
#that the user has at least one movie belonging to one of the clusters and then take the average of those movies
#this is exactly like a later training step excpet it is applied to all the movies the user watched

#for a single randomly chosen movie from each user in the trainging data...

#find the cluster the movie belongs to 
#find the movies part of that same cluster that the user has scored at the highest possible k value
#take the average score of these movies
#find the cluster the user belongs to
#find the average rating of the movie for users in that cluster at the highest possible k value
#train an mlp model with both averages and perhaps some extra statistics as features...
#using the given movie ratings as actuals


#The process of predicting a rating:
#1. find the cluster the movie belongs to 
#2. find the movies part of that same cluster that the user has scored at the highest possible k value
#3. take the average score of these movies
#4. find the cluster the user belongs to
#5. find the average rating of the movie for users in that cluster at the highest possible k value
#6. input into the trained mlp model both averages and perhaps some extra statistics
#7. make predictions and test against the randomly chosen movies actual ratings


#summary:
#find cluster for movie -> find movies part of the same clusters that the users rated -> average
#question: are the clusters unique to the movies the user has watched or to all movies???
#what is the technical difference???
#is this the same as finding the most simimlair movie the user rated and copying the rating???

#find cluster for user -> find the ratings for the movie by people in the same cluster -> average

#other avenues considered:
#idea 1:
#for the first process, instead of averaging the movies that only the user rated, find other users that are...
#like the user in question and find the average for that movie cluster
#Problem: it is better to get the users raw opionion rather than generalizing it to some like minded users
#there is an extra costly step to this
#idea 2: 
#for the second process, instead of finding the average rating for the movie in the same cluster of users...
#also find the average rating of movies that are like the movie in question 
#Problem, it is better to get the movies rating itself as it would be the most accurate indicator
#there is an extra costly step to this




#test with overall averges:
#note: users have between 50 and 75 ratings each
#note: there are 250 users who are taken into account 
#note: seed int is one for above cells (cells part of creating the csv file)

#with seed int == 3, 4 taking overall averages: 0.09565753948597455
#with seed_int == 1, 2 taking overall averages: 0.070404868516315
#with seed_int == 2, 5 taking overall averages: 0.11310085954932936
#with seed_int == 4, 6 taking overall averages: 0.07125374341347135
#with seed_int == 5, 4 taking overall averages: 0.17736444913943628
#compute time: 11 minutes


#test with users related movies:
#is there a magic proportion of movies to average???
#note: this is taking around the same time as the above tests meaning 
#there could be more users to include in analysis with little increase in runtime
#k fold cross validation could be effective
#https://www.youtube.com/watch?v=TIgfjmp-4BA&ab_channel=Udacity

#effect of choosing a random seed...
#https://towardsdatascience.com/how-to-use-random-seeds-effectively-54a4cd855a79

#try tinkering with the number of similair movies to average
